In [98]:
# Freesound API

In [99]:
import freesound
from requests_oauthlib import OAuth2Session
import os
import time
import pickle
from requests_ratelimiter import LimiterSession

In [101]:


authorization_base_url = 'https://freesound.org/apiv2/oauth2/authorize/'
token_url = 'https://freesound.org/apiv2/oauth2/access_token/'

In [103]:
client_id = "" # grab from fs_config.py
client_secret = "" # grab from fs_config.py

In [106]:
# do the OAuth dance
oauth = OAuth2Session(client_id)

authorization_url, state = oauth.authorization_url(
    "https://freesound.org/apiv2/oauth2/authorize/"
)
print(f"Please go to {authorization_url} and authorize access.")

authorization_code = input("Please enter the authorization code:")
oauth_token = oauth.fetch_token(
    "https://freesound.org/apiv2/oauth2/access_token/",
    authorization_code,
    client_secret=client_secret,
)

# oauth_token
client = freesound.FreesoundClient()
client.set_token(oauth_token["access_token"], "oauth")

Please go to https://freesound.org/apiv2/oauth2/authorize/?response_type=code&client_id=7yB3RiAb8ipn6BQuwvyn&state=a4QYiqqn1YgRRdrLJRyfspwyNzvFtL and authorize access.


In [107]:
# Get sound info example
print("Sound info:")
print("-----------")
sound = client.get_sound(96541)
print("Getting sound:", sound.name)
print("Url:", sound.url)
print("Description:", sound.description)
print("Tags:", " ".join(sound.tags))
print()

Sound info:
-----------
Getting sound: coming soon.wav
Url: https://freesound.org/people/tim.kahn/sounds/96541/
Description: For the user "Stealth Inc.", my girlfriend saying "Coming soon". Recorded with an AT2020 microphone into an Apogee Duet and edited with Record.
Tags: request female girl talk vocal voice american woman english speak



In [111]:
# for simplicity, lets start with all MTG sounds - very clean, usually single instruments
# search for MTG
user = client.get_user("MTG")
# print(dir(user))
print(user.num_sounds)
user_sounds = user.get_sounds(page_size = 10,fields = "id,name,tags,created,license,type,channels,filesize,bitrate,bitdepth,duration,samplerate,username",
                              sort = 'rating_desc')
# download wave files
# create dict for file name and tags/analysis
# start building the model!!

# in the future, we can download 'messier' files...

8117


In [112]:
num_pages = user.num_sounds // 10 + 1

num_pages

812

In [114]:
# user_sounds
# page_num = 1
# while page_num < num_pages:
#     for sound in user_sounds:
#         print("\t-", sound.name, "by", sound.username, "tags", sound.tags)
#     time.sleep(5)
#     user_sounds = user_sounds.next_page()
#     # increment page
#     page_num += 1

In [116]:
# your_directory_path = "your_directory_path"

# subdirectory_names = []

# for root, dirs, files in os.walk("./freesound"):
#     for directory in dirs:
#         subdirectory_names.append(directory)

# # Print all collected subdirectory names
# len(subdirectory_names)

In [136]:
# dir(user_sounds[0])

def fs_download(sound):
    """For a given freesound sound object, download audio file + metadata + analysis"""
    # basic info
    sound_info = sound.as_dict()

    # download sound:
    # get basic info
    # check if sound is in file system
    # downloaded_sounds = [x[0] for x in os.walk("./freesound")]
    downloaded_sounds = []

    for root, dirs, files in os.walk("./freesound"):
        for directory in dirs:
            downloaded_sounds.append(directory)
    # if in file_system:
    if str(sound_info['id']) in downloaded_sounds:
        # r"C:\Users\{}\Documents\file.txt".format(username)
        # skip
        print('folder already created. skipping')
        pass
    else:
        # make directory
        sound_dir = f"./freesound/{sound_info['id']}"
        os.mkdir(sound_dir)
        
        # save basic info
        info_pkl = f"{sound_dir}/sound_metadata.pkl"
        with open(info_pkl, 'wb') as f:
            pickle.dump(sound_info, f)
        # save analysis
        sound_analysis = sound.get_analysis().as_dict()
        analysis_pkl = f"{sound_dir}/sound_analysis.pkl"
        with open(analysis_pkl,'wb') as f:
            pickle.dump(sound_analysis,f)
            
        # download
        name_string = f"{sound_info['name']}.{sound_info['type']}"
        name_string = name_string.replace('\r', '')
        sound.retrieve(directory=sound_dir,name = name_string)
        

In [118]:
from requests_ratelimiter import LimiterSession

# Apply a rate-limit (59 requests per minute) to all requests
client.session = LimiterSession(per_minute=59)

In [119]:
# for simplicity, lets start with all MTG sounds - very clean, usually single instruments
# search for MTG
user = client.get_user("MTG")
# print(dir(user))
print(user.num_sounds)
user_sounds = user.get_sounds(page_size = 100,fields = "id,name,tags,created,license,type,channels,filesize,bitrate,bitdepth,duration,samplerate,username",
                              sort = 'rating_desc')


8117


In [120]:
num_pages = user.num_sounds // 10 + 1
print(num_pages)
# num_pages = 2 # temp for testing

812


In [137]:
page_num = 1
while page_num < num_pages:
    for sound in user_sounds:
        print("\t-", sound.name, "by", sound.username, "tags", sound.tags)
        fs_download(sound)
    time.sleep(5)
    tries = 0
    while tries < 10:
        try:
            user_sounds = user_sounds.next_page()
            tries = 10
        except freesound.FreesoundException as e:
            print('bad gateway, trying again')
            tries += 1
            time.sleep(3)
    
    # increment page
    page_num += 1

 by MTG tags ['Fsharp4', 'single-note', 'clarinet', 'multisample', 'good-sounds', 'neumann-U87']
folder already created. skipping
 by MTG tags ['F4', 'single-note', 'clarinet', 'good-sounds', 'multisample', 'neumann-U87']
folder already created. skipping
 by MTG tags ['E4', 'single-note', 'clarinet', 'good-sounds', 'multisample', 'neumann-U87']
folder already created. skipping
 by MTG tags ['Dsharp4', 'single-note', 'neumann-U87', 'clarinet', 'good-sounds', 'multisample']
folder already created. skipping
 by MTG tags ['clarinet', 'D4', 'single-note', 'good-sounds', 'multisample', 'neumann-U87']
folder already created. skipping
 by MTG tags ['Csharp4', 'single-note', 'clarinet', 'good-sounds', 'multisample', 'neumann-U87']
folder already created. skipping
 by MTG tags ['single-note', 'clarinet', 'C4', 'good-sounds', 'multisample', 'neumann-U87']
folder already created. skipping
 by MTG tags ['B3', 'single-note', 'clarinet', 'good-sounds', 'multisample', 'neumann-U87']
folder already cre

FreesoundException: <FreesoundException: code=429, detail="Too Many Requests">

In [ ]:
# audit
# next steps:
# add relevant directories to gitignore
# add git ignore file to github dknapp17/audio_ML
# manually add config and notebooks and requirements.txt to github
# write out workflow for freesound API and running freesound notebook
# document notebook, add cool visuals
# resubmit to github